In [4]:
!pip install sklearn_crfsuite


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [6]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/DataNer.txt", sep=" ", header=None, skip_blank_lines=False, encoding="utf-8")
data.columns = ["word","pos", "upper", "number","ner"]
tag_distribution = data.groupby("ner").size().reset_index(name='counts')
print(tag_distribution)
classes = list(filter(lambda x: x not in ["O", np.nan], list(data["ner"].unique())))

print(classes)
sentences, sentence = [], []
# For each row in the NER data...
for index, row in data.iterrows():
    # If the row is empty (no string in the token column)
    if type(row["word"]) != str:
        # If the current sentence is not empty, append it to the sentences and create a new sentence
        if len(sentence) > 0:
            sentences.append(sentence)
            sentence = []
    # Otherwise...
    else:
        # If the row does not indicate the start of a document, add the token to the current sentence
        if type(row["word"]) != float and type(row["pos"]) != float and type(row["ner"]) != float:
            if not row["word"].startswith("-DOCSTART-"):
                sentence.append([row["word"], row["pos"], row["upper"],row["number"] ,row["ner"]])
    # pbar.update()
print(len(sentences))

# data = data.fillna(method="ffill")
# data.tail(50)

                 ner  counts
0   B-Admission_Type     192
1      B-Campus_Name     119
2        B-Dept_Name      39
3        B-Docs_Name     171
4          B-K_years      30
5       B-Major_Mode     241
6       B-Major_Name     740
7     B-Scholar_Name     219
8     B-Subject_Name     120
9     B-Teacher_Name      38
10        B-Uni_Name     952
11        B-datetime     509
12        B-duration      80
13        B-location       1
14          B-number     197
15  I-Admission_Type     333
16     I-Campus_Name     119
17       I-Dept_Name      47
18       I-Docs_Name     437
19         I-K_years      41
20      I-Major_Mode     475
21      I-Major_Name    1114
22    I-Scholar_Name     537
23    I-Subject_Name      94
24    I-Teacher_Name      40
25        I-Uni_Name     708
26        I-datetime     531
27        I-duration     103
28          I-number       1
29                 O   31968
['B-Subject_Name', 'I-Subject_Name', 'B-number', 'B-datetime', 'I-datetime', 'B-Uni_Name', 'I-Uni_Nam

In [7]:
sentences[2]

[['Trong', 'E', 0.0, 0.0, 'O'],
 ['bài', 'N', 0.0, 0.0, 'O'],
 ['thi', 'V', 0.0, 0.0, 'O'],
 ['tiếng', 'N', 0.0, 0.0, 'B-Subject_Name'],
 ['anh', 'Nc', 0.0, 0.0, 'I-Subject_Name'],
 ['kỹ_năng', 'N', 0.0, 0.0, 'O'],
 ['nói', 'V', 0.0, 0.0, 'O'],
 ['chiếm', 'V', 0.0, 0.0, 'O'],
 ['bao_nhiêu', 'P', 0.0, 0.0, 'O'],
 ['điểm', 'N', 0.0, 0.0, 'O'],
 ['ạ', 'I', 0.0, 0.0, 'O'],
 ['?', 'CH', 0.0, 0.0, 'O']]

In [8]:
words = set(list(data["word"].values))
words.add('PADword')
n_words = len(words)
print(n_words)

2164


In [9]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag,upper,number,label in sent]

def sent2tokens(sent):
    return [token for token, postag,upper,number, label in sent]



In [10]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [12]:
!pip install sklearn

In [13]:

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [14]:
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [134]:
import  pickle

In [136]:
filename = '/content/drive/My Drive/Colab Notebooks/NLP/models/model_CRF_NER.pkl'
pickle.dump(crf, open(filename, 'wb'))
print("SAVED SUCESS")

SAVED SUCESS


In [15]:
#  !pip install sklearn
#  from sklearn.metrics import f1_score

In [16]:

y_pred = crf.predict(X_test)

In [37]:
y_pred[1]

['O', 'O', 'O', 'O', 'O', 'O', 'O']

In [17]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9600983915240606


In [18]:
report = flat_classification_report(y_test, y_pred)
print(report)

                  precision    recall  f1-score   support

B-Admission_Type       0.95      0.88      0.91        42
   B-Campus_Name       0.90      0.83      0.86        23
     B-Dept_Name       1.00      0.78      0.88         9
     B-Docs_Name       0.82      0.69      0.75        39
       B-K_years       1.00      0.86      0.92         7
    B-Major_Mode       0.82      0.75      0.78        55
    B-Major_Name       0.92      0.83      0.87       159
  B-Scholar_Name       0.83      0.98      0.90        41
  B-Subject_Name       0.90      0.83      0.86        23
  B-Teacher_Name       1.00      1.00      1.00         7
      B-Uni_Name       0.94      0.91      0.92       162
      B-datetime       0.89      0.95      0.92        86
      B-duration       0.94      0.79      0.86        19
      B-location       0.00      0.00      0.00         1
        B-number       0.85      0.95      0.90        42
I-Admission_Type       0.96      0.82      0.89        80
   I-Campus_N

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# crf = CRF(algorithm='lbfgs',
#           c1=10,
#           c2=0.1,
#           max_iterations=100,
#           all_possible_transitions=False)
# crf.fit(X_train, y_train)

In [20]:
# Predicting on the test set.
# y_pred = crf.predict(X_test)

In [21]:
# f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
# print(f1_score)

In [22]:
# report = flat_classification_report(y_test, y_pred)
# print(report)

In [92]:
test_sent = "Cho em hỏi học phí ngành kế toán ạ?"

In [114]:
!pip install underthesea
from underthesea import word_tokenize

In [93]:
from underthesea import pos_tag


In [94]:
pos = pos_tag(test_sent)

In [95]:
X = [sent2features(pos)]

In [116]:
sentence = word_tokenize(test_sent)

In [127]:
pred = crf.predict(X)
pred = np.array(pred) 

In [128]:
pred = pred.flatten()

In [133]:
pred

array(['O', 'O', 'O', 'O', 'B-Major_Name', 'I-Major_Name', 'O', 'O'],
      dtype='<U12')

In [118]:
def unique(list1): 
  
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 
    for x in unique_list: 
        return x

In [132]:
# Product perform by upper word
sentence2string = ''
i = 0

for word, label in list(zip(sentence, pred)):
  if label[0] == 'B':
    
    tag =[]
    sentence2string  = ''
    sentence2string += (word.upper() +' ')
    tag.append(label[2:])
  if label[0] == 'I':
    sentence2string += (word.upper() +' ')
  if label[0] == 'I' and pred[i+1][0] == 'O':
    print("{:15}: {:5}".format(sentence2string, unique(tag)))
  if label[0] == 'I' and pred[i+1][0] == 'B':
    print("{:15}: {:5}".format(sentence2string, unique(tag)))
  i = i+1  

NGÀNH KẾ TOÁN  : Major_Name
